This is a practice in building various recommedation engine based on collaborative filtering using the movielens movie db

-item-item

-user-user

-user-item

-SVD/matrix transformation


In [10]:
import numpy as np
import pandas as pd
import re
import sklearn as skl
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [17]:

# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading ratings file:
r_cols = ['user_id', 'item_id', 'rating', 'unix_timestamp']
df = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.data', sep='\t', names=r_cols,
 encoding='latin-1')



In [19]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print( 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 943 | Number of movies = 1682


In [20]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

C:\Users\suhaib.qazi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [24]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [25]:
user_similarity

array([[ 0.        ,  0.91401167,  0.96781522, ...,  0.91089936,
         0.90125738,  0.71699015],
       [ 0.91401167,  0.        ,  0.98003912, ...,  0.88171402,
         0.92863619,  0.89550852],
       [ 0.96781522,  0.98003912,  0.        , ...,  0.97428557,
         0.92076815,  0.98114175],
       ..., 
       [ 0.91089936,  0.88171402,  0.97428557, ...,  0.        ,
         1.        ,  0.95714059],
       [ 0.90125738,  0.92863619,  0.92076815, ...,  1.        ,
         0.        ,  0.91497037],
       [ 0.71699015,  0.89550852,  0.98114175, ...,  0.95714059,
         0.91497037,  0.        ]])

In [30]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine',algorithm='brute')
item_knn = NearestNeighbors(metric='cosine',algorithm='brute')

knn.fit(train_data_matrix)
knn.fit(train_data_matrix.T)

#model_knn.fit(rating_items_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [33]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [34]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [35]:
item_prediction

array([[ 0.36439027,  0.37670476,  0.39651342, ...,  0.43700367,
         0.42663515,  0.43426532],
       [ 0.07555684,  0.08830118,  0.08494649, ...,  0.08838388,
         0.08967995,  0.08982748],
       [ 0.0725308 ,  0.07505615,  0.0724635 , ...,  0.0691084 ,
         0.07293585,  0.07317073],
       ..., 
       [ 0.02308044,  0.02971715,  0.02859047, ...,  0.03268411,
         0.03205027,  0.03271862],
       [ 0.12079782,  0.12787999,  0.13443713, ...,  0.13904414,
         0.13741215,  0.13920286],
       [ 0.21142065,  0.21222886,  0.22757909, ...,  0.26390345,
         0.25596299,  0.26174896]])